In [5]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_core.models import ModelInfo
from autogen_agentchat.conditions import MaxMessageTermination

In [6]:
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor

In [7]:
from autogen_ext.agents.file_surfer import FileSurfer
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from autogen_ext.agents.video_surfer import VideoSurfer
from autogen_ext.agents.magentic_one import MagenticOneCoderAgent

In [8]:
from autogen_agentchat.teams import RoundRobinGroupChat, MagenticOneGroupChat

In [9]:
from autogen_agentchat.ui import Console

In [10]:
termination_condition = MaxMessageTermination(3)

In [11]:
vllm_model = OpenAIChatCompletionClient(
        model="Team-ACE/ToolACE-2-Llama-3.1-8B",
        base_url="https://g3.ai.qylis.com/tllama3/v1",
        api_key="NotRequired",
        model_info=ModelInfo(vision=False, function_calling=True, json_output=True, family="llama", structured_output=True),
    )

agent = AssistantAgent(
        name="chat_agent",
        model_client=vllm_model, 
    )

In [12]:
result = await agent.run(task="Find information on Mexico")
print(result.messages)

[TextMessage(source='user', models_usage=None, metadata={}, content='Find information on Mexico', type='TextMessage'), TextMessage(source='chat_agent', models_usage=RequestUsage(prompt_tokens=43, completion_tokens=497), metadata={}, content="Mexico is a country located in North America, known for its rich cultural heritage, diverse geography, and vibrant cities. Here are some key points about Mexico:\n\n1. **Location and Geography**: Mexico is bordered by the United States to the north, Belize and Guatemala to the southeast, and the Pacific Ocean to the west and the Gulf of Mexico to the east. It has a diverse geography, including deserts, mountains, and coastlines.\n\n2. **Population and Capital**: As of 2021, Mexico's population is approximately 127 million. The capital city is Mexico City, which is one of the largest cities in the world and a major cultural and economic center.\n\n3. **Language**: The official language is Spanish, and it is the most widely spoken language in the cou

In [ ]:
from autogen_core.tools import FunctionTool


# Define a tool using a Python function.
async def web_search_func(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."

# This step is automatically performed inside the AssistantAgent if the tool is a Python function.
web_search_function_tool = FunctionTool(web_search_func, description="Find information on the web")
# The schema is provided to the model during AssistantAgent's on_messages call.
web_search_function_tool.schema

In [ ]:
tool_agent = AssistantAgent(
        name="chat_agent",
        model_client=vllm_model, 
        tools=[web_search_function_tool]
    )

In [ ]:
result = await tool_agent.run(task="Find information on AutoGen")
print(result.messages)

In [ ]:
file_agent = FileSurfer(
        name="file_surfer",
        model_client=vllm_model,
    )

In [ ]:
result = await file_agent.run(task="How many .json files are in the current directory. List their paths.")
print(str(result.messages))

In [ ]:
result = await file_agent.run(task="What are the contents of chat-tool.json file.")
print(str(result.messages))

In [ ]:
web_surfer_agent = MultimodalWebSurfer(
        name="web_agent",
        model_client=vllm_model,
    )

In [ ]:
# Define a team
agent_team = RoundRobinGroupChat([web_surfer_agent], max_turns=3)

In [ ]:
# Run the team and stream messages to the console
stream = agent_team.run_stream(task="Navigate to the microsoft readme on GitHub. Try again")
await Console(stream)
# Close the browser controlled by the agent
await web_surfer_agent.close()

In [ ]:
# Define an agent
video_agent = VideoSurfer(
    name="VideoSurfer",
    model_client=vllm_model
    )

In [ ]:
# Define a team
agent_team = MagenticOneGroupChat([web_surfer_agent, video_agent], model_client=vllm_model,)

# Run the team and stream messages to the console
stream = agent_team.run_stream(task="Find a latest video about magentic one on youtube and extract quotes from it that make sense.")
await Console(stream)

In [25]:
coder_agent = MagenticOneCoderAgent(
    name="coder_agent",
    model_client=vllm_model,
)

In [26]:
# define the Docker CLI Code Executor
code_executor = DockerCommandLineCodeExecutor(work_dir="coding")

In [27]:
await code_executor.start()

In [28]:
code_executor_agent = CodeExecutorAgent("code_executor_agent", code_executor=code_executor)

In [30]:
groupchat = RoundRobinGroupChat(
    participants=[coder_agent, code_executor_agent], termination_condition=termination_condition
)

task = "Write python code to print Hello World! with the correct markdown format"
await Console(groupchat.run_stream(task=task))

---------- user ----------
Write python code to print Hello World! with the correct markdown format
---------- coder_agent ----------
```python
# Print "Hello World!" in markdown format
print("Hello World!")
```
---------- code_executor_agent ----------
Hello World!



TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write python code to print Hello World! with the correct markdown format', type='TextMessage'), TextMessage(source='coder_agent', models_usage=RequestUsage(prompt_tokens=487, completion_tokens=20), metadata={}, content='```python\n# Print "Hello World!" in markdown format\nprint("Hello World!")\n```', type='TextMessage'), TextMessage(source='code_executor_agent', models_usage=None, metadata={}, content='Hello World!\n', type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')

In [ ]:
# stop the execution container
await code_executor.stop()